In [ ]:
################################################################################################################################################################################
library(Seurat)
library(SeuratObject)
library(ggplot2)
library(cowplot)
library(dplyr)

library(gridExtra)
#library(hash)
library(ggsci)
#library(presto)

library(CellChat)
#library(tidyverse)
library(NMF)
library(ggalluvial)
library(patchwork)
library(svglite)
library(Matrix)
library(rjson)
library(RColorBrewer)

library(ggrastr)#格栅化文件大小

library(ggdark)

library(stringr)
library(spacexr)
library(factoextra) # Kmean 聚类前的一些判断和统计

suppressMessages(library(data.table))
suppressMessages(library(Matrix))
suppressMessages(library(rjson))


In [ ]:
Normalization_Clustering <- function(sce, Normalization = TRUE, dims = 20, resolution = 0.1){
    if (Normalization){
       sce <- SCTransform(sce, assay = "RNA", return.only.var.genes = FALSE, n_genes=NULL, method='qpoisson')
    }
    sce <- RunPCA(sce)
    sce <- FindNeighbors(sce, dims = 1:dims)
    sce <- FindClusters(sce, verbose = FALSE, resolution = resolution)
    sce <- RunUMAP(sce, dims = 1:dims)
    sce <- RunTSNE(sce, dims = 1:dims)
  return(sce)
}

In [ ]:
gem_2_rds <- function(gem, prefix, binsize){
    #' group counts into bins
    data=fread(file=gem)
    colnames(data)=c("geneID","x","y","MIDCounts")
    data$x <- trunc(data$x / binsize) * binsize
    data$y <- trunc(data$y / binsize) * binsize
    #trunc取整函数，只取整数部分
    #将横纵坐标根据binsize取整，比如，如果binszie=10,x=2401,y=4522,
    #那么处理后，x=2400,y=4520
    
    if ('MIDCounts' %in% colnames(data)) {
#       <- aggregate(data$MIDCounts,by=list(data$geneID,data$x,data$y),FUN=sum)
      # aa %in% bb，判断aa是否存在于bb中
      data <- data[, .(counts=sum(MIDCounts)), by = .(geneID, x, y)]
    } else {
      data <- data[, .(counts=sum(UMICount)), by = .(geneID, x, y)]
    }
    
    #' create sparse matrix from stereo
    data$cell <- paste0(prefix, ':', data$x, '_', data$y)
    data$geneIdx <- match(data$geneID, unique(data$geneID))
    data$cellIdx <- match(data$cell, unique(data$cell))
    #match(aa,bb)返回bb在aa中的位置
    
    # if (! is.null(opts$binsize)){
    #   #is.null(aa) 判断aa是否为空，如果空，返回Ture；
    #   #! is.null(aa)，相反地，判断aa是否非空，如果非空，返回True；
    #   write.table(data, file = paste0(opts$outdir, '/', opts$sample, '_bin', opts$binsize, '.tsv'), 
    #               quote = FALSE, sep = '\t', row.names = FALSE)
    # }
    #write.table(aa,file=xx,sep =" ",quote=TRUE,row.names =TRUE, col.names =TRUE)
    #把内容aa输出到文件xx中
    
    mat <- sparseMatrix(i = data$geneIdx, j = data$cellIdx, x = data$counts, 
                        dimnames = list(unique(data$geneID), unique(data$cell)))
    #sparseMatrix稀疏矩阵函数
    
    cell_coords <- unique(data[, c('cell', 'x', 'y')])
    #unique去除重复函数，删除cell,x和y都一样的行
    
    rownames(cell_coords) <- cell_coords$cell
    
    #cell_coords$cell <- NULL
    
    seurat_spatialObj <- CreateSeuratObject(counts = mat, project = 'Stereo', assay = 'Spatial', 
                                            names.delim = ':', meta.data = cell_coords,min.features=50)
    
    
    #' create pseudo image
    cell_coords$x <- cell_coords$x - min(cell_coords$x) + 1
    cell_coords$y <- cell_coords$y - min(cell_coords$y) + 1
    
    tissue_lowres_image <- matrix(1, max(cell_coords$y), max(cell_coords$x))
    #matrix(aa,x,y)以aa为输入向量，创建一个x行y列的矩阵
    #构造一个seruat image
    
    tissue_positions_list <- data.frame(row.names = cell_coords$cell,
                                        tissue = 1,
                                        row = cell_coords$y, col = cell_coords$x,
                                        imagerow = cell_coords$y, imagecol = cell_coords$x)
    
    
    scalefactors_json <- toJSON(list(fiducial_diameter_fullres = binsize,
                                     tissue_hires_scalef = 1,
                                     tissue_lowres_scalef = 1))
    #toJSON: 把json格式 转换成 list格式
    
    
    #' function to create image object
    generate_spatialObj <- function(image, scale.factors, tissue.positions, filter.matrix = TRUE){
      if (filter.matrix) {
        tissue.positions <- tissue.positions[which(tissue.positions$tissue == 1), , drop = FALSE]
      }
      
      unnormalized.radius <- scale.factors$fiducial_diameter_fullres * scale.factors$tissue_lowres_scalef
      
      spot.radius <- unnormalized.radius / max(dim(x = image))
      
      return(new(Class = 'VisiumV1', 
                 image = image, 
                 scale.factors = scalefactors(spot = scale.factors$tissue_hires_scalef, 
                                              fiducial = scale.factors$fiducial_diameter_fullres, 
                                              hires = scale.factors$tissue_hires_scalef, 
                                              lowres = scale.factors$tissue_lowres_scalef), 
                 coordinates = tissue.positions, 
                 spot.radius = spot.radius))
    }
    
    spatialObj <- generate_spatialObj(image = tissue_lowres_image, 
                                      scale.factors = fromJSON(scalefactors_json), 
                                      tissue.positions = tissue_positions_list)
    #可以理解为构建一个spatial背景
    
    #' import image into seurat object
    spatialObj <- spatialObj[Cells(x = seurat_spatialObj)]
    DefaultAssay(spatialObj) <- 'Spatial'
    
    seurat_spatialObj[['slice1']] <- spatialObj
    rm("spatialObj")
    rm("data")
    rm("mat")
    
    return(seurat_spatialObj)
    }  

In [ ]:
rotation_coord<-function(object,numerator,denominator){
    #旋转120度,numerator设置为2,denominator设置为3,2*pi/3=120
    #如果不旋转,numerator设置为2,denominator设置为1,2*pi/1=360
    mat1<-data.frame(object$x,object$y)
    mat1<-as.matrix(mat1)
    mat2 <- matrix(c(cos(numerator*pi/denominator),sin(numerator*pi/denominator),-sin(numerator*pi/denominator),cos(numerator*pi/denominator)), nrow=2, ncol=2)
    coord <- mat1 %*% mat2
    object$old_x<-object$x
    object$old_y<-object$y
    object$x<-coord[,1]
    object$y<-coord[,2]
    return(object) 
}

In [ ]:
#gen_exp_filter=function(rds,gene=c("Acta2","Myl9"),outdir,prefix,cut){
gen_exp_filter=function(rds,gene=c("Acta2","Myl9"),cut){
    
    #rds=gf_ile1
    #gene=c("F3","Sox6","Cxcl14","Wnt5a","Bmp7")
    #gene=c("Gpm6a","Upk3b","Msln","Bcam","Lrrn4")
    #cut=0.5
    #gene=c("Ighg3","Ighg2b","Ighg2c","Ighg1","Igha")
    
    library(stringr)
    gen=FetchData(rds,vars=gene)
    nam=rownames(gen)
    gen=apply(gen,1,sum)
   
    x=as.numeric(str_split_fixed(nam, "[:_]", 3)[,2]) #GF:13750_6550,格式必须一致
    y=as.numeric(str_split_fixed(nam, "[:_]", 3)[,3]) #GF:13750_6550,格式必须一致
    
    new=data.frame(x,y,gen)
    #write.table(new,file=paste(outdir,"/",prefix,".txt",sep=""),row.names=F,col.names=T,sep="\t")
    colnames(new)=c("x","y","exp")
    pos=which(new$exp==0)
    if(length(pos)>0){
        new=new[-pos,]
        pp=which(new$exp>=cut)
        new$exp=0
        new[pp,3]=1
        pp=which(new$exp==0)
        if(length(pp)>0){
            new=new[-pp,]
        }
    }
    res=as.matrix(dist(new, method = "euclidean", diag = T, upper =T, p = 2))
    min_dis=c()
    #names(min_dis)=rownames(res)
    for(i in 1:dim(res)){
        nam=colnames(res)
        val=c()
        tmp=res[i,]
        p1=which.min(tmp)
        val[1]=tmp[p1]
        tmp[p1]=100000
        p2=which.min(tmp)
        val[2]=tmp[p2]
        tmp[p2]=100000
        p3=which.min(tmp)
        val[3]=tmp[p3]
        tmp[p3]=100000
        p4=which.min(tmp)
        val[4]=tmp[p4]
        tmp[p4]=100000
        p5=which.min(tmp)
        val[5]=tmp[p5]
        tmp[p5]=100000
        p6=which.min(tmp)
        val[6]=tmp[p6]
        tmp[p6]=100000
        p7=which.min(tmp)
        val[7]=tmp[p7]
        tmp[p7]=100000
        p8=which.min(tmp)
        val[8]=tmp[p8]
        tmp[p8]=100000
        min_dis[i]=mean(val)
    }
    return(data.frame(rownames(res),min_dis))
}

In [ ]:
gen_score_filter=function(rds,genelist,cut){#rds需要标准化
    
    gen=FetchData(rds,vars=genelist)
    nam=rownames(gen)
    rds <- AddModuleScore(rds,features =list(colnames(gen)),name="score",nbin = 5)
   
    x=as.numeric(str_split_fixed(nam, "[:_]", 3)[,2]) #GF:13750_6550,格式必须一致
    y=as.numeric(str_split_fixed(nam, "[:_]", 3)[,3]) #GF:13750_6550,格式必须一致
    
    new=data.frame(x,y,rds@meta.data$score1)
    #write.table(new,file=paste(outdir,"/",prefix,".txt",sep=""),row.names=F,col.names=T,sep="\t")
    colnames(new)=c("x","y","score")
    pos=which(new$score<=0)
    if(length(pos)>0){
        new=new[-pos,]
        pp=which(new$score>cut)
        new$score=0
        new[pp,3]=1
        pp=which(new$score==0)
        if(length(pp)>0){
            new=new[-pp,]
        }
    }
    res=as.matrix(dist(new, method = "euclidean", diag = T, upper =T, p = 2))
    min_dis=c()
    #names(min_dis)=rownames(res)
    for(i in 1:dim(res)[1]){
        nam=colnames(res)
        val=c()
        tmp=res[i,]
        p1=which.min(tmp)
        val[1]=tmp[p1]
        tmp[p1]=100000
        p2=which.min(tmp)
        val[2]=tmp[p2]
        tmp[p2]=100000
        p3=which.min(tmp)
        val[3]=tmp[p3]
        tmp[p3]=100000
        p4=which.min(tmp)
        val[4]=tmp[p4]
        tmp[p4]=100000
        p5=which.min(tmp)
        val[5]=tmp[p5]
        tmp[p5]=100000
        p6=which.min(tmp)
        val[6]=tmp[p6]
        tmp[p6]=100000
        p7=which.min(tmp)
        val[7]=tmp[p7]
        tmp[p7]=100000
        p8=which.min(tmp)
        val[8]=tmp[p8]
        tmp[p8]=100000
        min_dis[i]=mean(val)
    }
    return(data.frame(rownames(res),min_dis)) # 返回的矩阵的index，这个index的引用必须要保证new在前后都一致
}

In [ ]:
gen_score_dis=function(rds,genelist,cut){#rds需要标准化
    
    gen=FetchData(rds,vars=genelist)
    nam=rownames(gen)
    rds <- AddModuleScore(rds,features =list(colnames(gen)),name="score",nbin = 5)
   
    x=as.numeric(str_split_fixed(nam, "[:_]", 3)[,2]) #GF:13750_6550,格式必须一致
    y=as.numeric(str_split_fixed(nam, "[:_]", 3)[,3]) #GF:13750_6550,格式必须一致
    
    new=data.frame(x,y,rds@meta.data$score1)
    #write.table(new,file=paste(outdir,"/",prefix,".txt",sep=""),row.names=F,col.names=T,sep="\t")
    colnames(new)=c("x","y","score")
    pos=which(new$score<=0)
    if(length(pos)>0){
        new=new[-pos,]
        pp=which(new$score>cut)
        new$score=0
        new[pp,3]=1
        pp=which(new$score==0)
        if(length(pp)>0){
            new=new[-pp,]
        }
    }
    return(new)
}
gen_rawscore_dis=function(rds,genelist){#rds需要标准化
    
    gen=FetchData(rds,vars=genelist)
    nam=rownames(gen)
    rds <- AddModuleScore(rds,features =list(colnames(gen)),name="score",nbin = 5)
   
    x=as.numeric(str_split_fixed(nam, "[:_]", 3)[,2]) #GF:13750_6550,格式必须一致
    y=as.numeric(str_split_fixed(nam, "[:_]", 3)[,3]) #GF:13750_6550,格式必须一致
    
    new=data.frame(x,y,rds@meta.data$score1)
    #write.table(new,file=paste(outdir,"/",prefix,".txt",sep=""),row.names=F,col.names=T,sep="\t")
    colnames(new)=c("x","y","score")
    pos=which(new$score<=0)
    if(length(pos)>0){
        new=new[-pos,]
    }
    return(new)
}

In [ ]:
score_area_size=function(dat,bin=20){#dat:"x","y","exp"，注意此处是非零的表达量，
    
    res=as.matrix(dist(dat, method = "euclidean", diag = T, upper =T, p = 2))
    res=replace(res,res!=bin,0) 
    res=replace(res,res==bin,1)#将紧邻的标记为1，非紧邻的标记为0
    
    test_sum=apply(res,1,sum)#过滤掉单独的孤零零的点，保证至少2个点紧邻
    pos=which(test_sum==0)
    if(length(pos)>0){
        res=res[-pos,-pos]
    }
    g=graph.adjacency(res,mode="undirected",weighted=NULL)
    clu <- components(g)### 基于近邻矩阵的聚类结果和bfs算法得出的结果是相同的
    grp=groups(clu)
    return(grp)
    #tmp=bfs(g,root=1,order=TRUE, rank=TRUE, father=TRUE, pred=TRUE,succ=TRUE, dist=TRUE)
    #plot(g)
    #fath=names(which(is.na(tmp$father)))
    #return(res)   
}


In [ ]:
density_area_size=function(den,grp){ # den是用gen_exp_filter(rds,genelist,nameflag)计算返回的结果；grp是score_area_size函数返回的结果
    
    rownames(den)=den[,1]
    min_den=c()
    avg_den=c()
    pro=c()
    
    for(i in 1:dim(grp)[1]){
        tmp=grp[[i]]
        min_den[i]=min(den[tmp,][,2])
        avg_den[i]=mean(den[tmp,][,2])
        pro[i]=length(tmp)
    }   
    return(data.frame(pro,min_den,avg_den))
    
}

In [ ]:
low_percentage_identify=function(raw_score,low_cut){##gf_raw_score: x,y,socre
    sor=sort(raw_score)
    index=round(length(raw_score)*low_cut)
    return(sor[index])
    
}


In [ ]:
gf_ile1 <- readRDS(file="/data/work/4.st_data/gf_ile1_bin20.rds")
gf_ile2 <- readRDS(file="/data/work/4.st_data/gf_ile2_bin20.rds")

spf_ile2 <- readRDS(file="/data/work/4.st_data/spf_ile2_bin20.rds")
spf_ile3 <- readRDS(file="/data/work/4.st_data/spf_ile3_bin20.rds")

gf_col1 <- readRDS(file="/data/work/4.st_data/gf_col1_bin20.rds")
spf_col1 <- readRDS(file="/data/work/4.st_data/spf_col1_bin20.rds")

gf_cem <- readRDS(file="/data/work/4.st_data/gf_cem_bin20.rds")
spf_cem <- readRDS(file="/data/work/4.st_data/spf_cem_bin20.rds")

gf_spl<- readRDS("/data/work/4.st_data/GF_spleen_bin20.rds")
spf_spl<- readRDS(file="/data/work/4.st_data/SPF_spleen_bin20.rds")

gf_thy<-readRDS(file="/data/work/4.st_data/GF_thymus_bin20.rds")
spf_thy<-readRDS(file="/data/work/4.st_data/SPF_thymus_bin20.rds")

gf_liv<-readRDS(file="/data/work/4.st_data/GF_liver_bin20.rds")
spf_liv<-readRDS(file="/data/work/4.st_data/SPF_liver_bin20.rds")


In [ ]:
plasma_gene=c("Jchain","Ighg3","Ighg2b","Ighg2c","Ighg1","Igha")
#plasma_gene=c("Jchain","Ighg3","Ighg2b","Ighg2c","Ighg1","Igha")

gf_col1_rawpla=gen_rawscore_dis(gf_col1,genelist=plasma_gene)
spf_col1_rawpla=gen_rawscore_dis(spf_col1,genelist=plasma_gene)
gf_ile1_rawpla=gen_rawscore_dis(gf_ile1,genelist=plasma_gene)
gf_ile2_rawpla=gen_rawscore_dis(gf_ile2,genelist=plasma_gene)
spf_ile2_rawpla=gen_rawscore_dis(spf_ile2,genelist=plasma_gene)
spf_ile3_rawpla=gen_rawscore_dis(spf_ile3,genelist=plasma_gene)
gf_cem_rawpla=gen_rawscore_dis(gf_cem,genelist=plasma_gene)
spf_cem_rawpla=gen_rawscore_dis(spf_cem,genelist=plasma_gene)

spf_liv_rawpla=gen_rawscore_dis(spf_liv,genelist=plasma_gene)
gf_liv_rawpla=gen_rawscore_dis(gf_liv,genelist=plasma_gene)

gf_thy_rawpla=gen_rawscore_dis(gf_thy,genelist=plasma_gene)
spf_thy_rawpla=gen_rawscore_dis(spf_thy,genelist=plasma_gene)


gf_spl_rawpla=gen_rawscore_dis(gf_spl,genelist=plasma_gene)
spf_spl_rawpla=gen_rawscore_dis(spf_spl,genelist=plasma_gene)

cem_cut=low_percentage_identify(c(gf_cem_rawpla[,3],spf_cem_rawpla[,3]),low_cut=0.05)
col_cut=low_percentage_identify(c(gf_col1_rawpla[,3],spf_col1_rawpla[,3]),low_cut=0.05)
ile_cut=low_percentage_identify(c(gf_ile1_rawpla[,3],gf_ile2_rawpla[,3],spf_ile2_rawpla[,3],spf_ile3_rawpla[,3]),low_cut=0.05)
thy_cut=low_percentage_identify(c(gf_thy_rawpla[,3],spf_thy_rawpla[,3]),low_cut=0.05)
spl_cut=low_percentage_identify(c(gf_spl_rawpla[,3],spf_spl_rawpla[,3]),low_cut=0.05)
liv_cut=low_percentage_identify(c(gf_liv_rawpla[,3],spf_liv_rawpla[,3]),low_cut=0.05)

In [ ]:
col_cut
ile_cut
cem_cut
thy_cut
spl_cut
liv_cut

In [ ]:
plasma_gene=c("Jchain","Ighg3","Ighg2b","Ighg2c","Ighg1","Igha")
gf_ile1_pla=gen_score_dis(gf_ile1,genelist=plasma_gene,cut=ile_cut)
gf_ile2_pla=gen_score_dis(gf_ile2,genelist=plasma_gene,cut=ile_cut)
spf_ile2_pla=gen_score_dis(spf_ile2,genelist=plasma_gene,cut=ile_cut)
spf_ile3_pla=gen_score_dis(spf_ile3,genelist=plasma_gene,cut=ile_cut)
gf_col1_pla=gen_score_dis(gf_col1,genelist=plasma_gene,cut=col_cut)
spf_col1_pla=gen_score_dis(spf_col1,genelist=plasma_gene,cut=col_cut)
gf_cem_pla=gen_score_dis(gf_cem,genelist=plasma_gene,cut=cem_cut)
spf_cem_pla=gen_score_dis(spf_cem,genelist=plasma_gene,cut=cem_cut)
spf_liv_pla=gen_score_dis(spf_liv,genelist=plasma_gene,cut=liv_cut)
gf_liv_pla=gen_score_dis(gf_liv,genelist=plasma_gene,cut=liv_cut)
gf_thy_pla=gen_score_dis(gf_thy,genelist=plasma_gene,cut=thy_cut)
spf_thy_pla=gen_score_dis(spf_thy,genelist=plasma_gene,cut=thy_cut)
gf_spl_pla=gen_score_dis(gf_spl,genelist=plasma_gene,cut=spl_cut)
spf_spl_pla=gen_score_dis(spf_spl,genelist=plasma_gene,cut=spl_cut)

gf_ile1_den=gen_score_filter(gf_ile1,genelist=plasma_gene,cut=ile_cut)
gf_ile2_den=gen_score_filter(gf_ile2,genelist=plasma_gene,cut=ile_cut)
spf_ile2_den=gen_score_filter(spf_ile2,genelist=plasma_gene,cut=ile_cut)
spf_ile3_den=gen_score_filter(spf_ile3,genelist=plasma_gene,cut=ile_cut)
gf_col1_den=gen_score_filter(gf_col1,genelist=plasma_gene,cut=col_cut)
spf_col1_den=gen_score_filter(spf_col1,genelist=plasma_gene,cut=col_cut)
gf_cem_den=gen_score_filter(gf_cem,genelist=plasma_gene,cut=cem_cut)
spf_cem_den=gen_score_filter(spf_cem,genelist=plasma_gene,cut=cem_cut)
spf_liv_den=gen_score_filter(spf_liv,genelist=plasma_gene,cut=liv_cut)
gf_liv_den=gen_score_filter(gf_liv,genelist=plasma_gene,cut=liv_cut)
gf_thy_den=gen_score_filter(gf_thy,genelist=plasma_gene,cut=thy_cut)
spf_thy_den=gen_score_filter(spf_thy,genelist=plasma_gene,cut=thy_cut)
gf_spl_den=gen_score_filter(gf_spl,genelist=plasma_gene,cut=spl_cut)
spf_spl_den=gen_score_filter(spf_spl,genelist=plasma_gene,cut=spl_cut)

gf_ile1_grp=score_area_size(gf_ile1_pla,bin=20)
gf_ile2_grp=score_area_size(gf_ile2_pla,bin=20)
spf_ile2_grp=score_area_size(spf_ile2_pla,bin=20)
spf_ile3_grp=score_area_size(spf_ile3_pla,bin=20)
gf_col1_grp=score_area_size(gf_col1_pla,bin=20)
spf_col1_grp=score_area_size(spf_col1_pla,bin=20)
gf_cem_grp=score_area_size(gf_cem_pla,bin=20)
spf_cem_grp=score_area_size(spf_cem_pla,bin=20)
gf_thy_grp=score_area_size(gf_thy_pla,bin=20)
spf_thy_grp=score_area_size(spf_thy_pla,bin=20)
gf_spl_grp=score_area_size(gf_spl_pla,bin=20)
spf_spl_grp=score_area_size(spf_spl_pla,bin=20)
spf_liv_grp=score_area_size(spf_liv_pla,bin=20)
gf_liv_grp=score_area_size(gf_liv_pla,bin=20)


gf_ile1_den_size=density_area_size(gf_ile1_den,gf_ile1_grp)
gf_ile2_den_size=density_area_size(gf_ile2_den,gf_ile2_grp)
spf_ile2_den_size=density_area_size(spf_ile2_den,spf_ile2_grp)
spf_ile3_den_size=density_area_size(spf_ile3_den,spf_ile3_grp)
gf_col1_den_size=density_area_size(gf_col1_den,gf_col1_grp)
spf_col1_den_size=density_area_size(spf_col1_den,spf_col1_grp)
gf_cem_den_size=density_area_size(gf_cem_den,gf_cem_grp)
spf_cem_den_size=density_area_size(spf_cem_den,spf_cem_grp)
gf_thy_den_size=density_area_size(gf_thy_den,gf_thy_grp)
spf_thy_den_size=density_area_size(spf_thy_den,spf_thy_grp)
gf_spl_den_size=density_area_size(gf_spl_den,gf_spl_grp)
spf_spl_den_size=density_area_size(spf_spl_den,spf_spl_grp)
gf_liv_den_size=density_area_size(gf_liv_den,gf_liv_grp)
spf_liv_den_size=density_area_size(spf_liv_den,spf_liv_grp)

In [ ]:
saveRDS(gf_ile1_grp,file="/data/work/7.immune_cell_aggregation_index/gf_ile1_grp.rds")
saveRDS(gf_ile2_grp,file="/data/work/7.immune_cell_aggregation_index/gf_ile2_grp.rds")

saveRDS(spf_ile2_grp,file="/data/work/7.immune_cell_aggregation_index/spf_ile2_grp.rds")
saveRDS(spf_ile3_grp,file="/data/work/7.immune_cell_aggregation_index/spf_ile3_grp.rds")

saveRDS(gf_col1_grp,file="/data/work/7.immune_cell_aggregation_index/gf_col1_grp.rds")
saveRDS(spf_col1_grp,file="/data/work/7.immune_cell_aggregation_index/spf_col1_grp.rds")

saveRDS(gf_cem_grp,file="/data/work/7.immune_cell_aggregation_index/gf_cem_grp.rds")
saveRDS(spf_cem_grp,file="/data/work/7.immune_cell_aggregation_index/spf_cem_grp.rds")

saveRDS(gf_thy_grp,file="/data/work/7.immune_cell_aggregation_index/gf_thy_grp.rds")
saveRDS(spf_thy_grp,file="/data/work/7.immune_cell_aggregation_index/spf_thy_grp.rds")

saveRDS(gf_spl_grp,file="/data/work/7.immune_cell_aggregation_index/gf_spl_grp.rds")
saveRDS(spf_spl_grp,file="/data/work/7.immune_cell_aggregation_index/spf_spl_grp.rds")

saveRDS(spf_liv_grp,file="/data/work/7.immune_cell_aggregation_index/spf_liv_grp.rds")
saveRDS(gf_liv_grp,file="/data/work/7.immune_cell_aggregation_index/gf_liv_grp.rds")



saveRDS(gf_ile1_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_ile1_den_size.txt")
saveRDS(gf_ile2_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_ile2_den_size.txt")

saveRDS(spf_ile2_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_ile2_den_size.txt")
saveRDS(spf_ile3_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_ile3_den_size.txt")

saveRDS(gf_col1_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_col1_den_size.txt")
saveRDS(spf_col1_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_col1_den_size.txt")

saveRDS(gf_cem_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_cem_den_size.txt")
saveRDS(spf_cem_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_cem_den_size.txt")

saveRDS(gf_thy_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_thy_den_size.txt")
saveRDS(spf_thy_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_thy_den_size.txt")

saveRDS(gf_spl_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_spl_den_size.txt")
saveRDS(spf_spl_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_spl_den_size.txt")

saveRDS(gf_liv_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_liv_den_size.txt")
saveRDS(spf_liv_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_liv_den_size.txt")

In [ ]:
macro=c("C1qa","C1qb","C1qc","Retnla","Pf4","Ms4a7","Mrc1","Cd68","Cd5l","Vsig4","Clec4f","Marco")

gf_col1_rawmac=gen_rawscore_dis(gf_col1,genelist=macro)
spf_col1_rawmac=gen_rawscore_dis(spf_col1,genelist=macro)
gf_ile1_rawmac=gen_rawscore_dis(gf_ile1,genelist=macro)
gf_ile2_rawmac=gen_rawscore_dis(gf_ile2,genelist=macro)
spf_ile2_rawmac=gen_rawscore_dis(spf_ile2,genelist=macro)
spf_ile3_rawmac=gen_rawscore_dis(spf_ile3,genelist=macro)
gf_cem_rawmac=gen_rawscore_dis(gf_cem,genelist=macro)
spf_cem_rawmac=gen_rawscore_dis(spf_cem,genelist=macro)

spf_liv_rawmac=gen_rawscore_dis(spf_liv,genelist=macro)
gf_liv_rawmac=gen_rawscore_dis(gf_liv,genelist=macro)

gf_thy_rawmac=gen_rawscore_dis(gf_thy,genelist=macro)
spf_thy_rawmac=gen_rawscore_dis(spf_thy,genelist=macro)

gf_spl_rawmac=gen_rawscore_dis(gf_spl,genelist=macro)
spf_spl_rawmac=gen_rawscore_dis(spf_spl,genelist=macro)

cem_mac_cut=low_percentage_identify(c(gf_cem_rawmac[,3],spf_cem_rawmac[,3]),low_cut=0.05)
col_mac_cut=low_percentage_identify(c(gf_col1_rawmac[,3],spf_col1_rawmac[,3]),low_cut=0.05)
ile_mac_cut=low_percentage_identify(c(gf_ile1_rawmac[,3],gf_ile2_rawmac[,3],spf_ile2_rawmac[,3],spf_ile3_rawmac[,3]),low_cut=0.05)
thy_mac_cut=low_percentage_identify(c(gf_thy_rawmac[,3],spf_thy_rawmac[,3]),low_cut=0.05)
spl_mac_cut=low_percentage_identify(c(gf_spl_rawmac[,3],spf_spl_rawmac[,3]),low_cut=0.05)
liv_mac_cut=low_percentage_identify(c(gf_liv_rawmac[,3],spf_liv_rawmac[,3]),low_cut=0.05)



In [ ]:
gf_ile1_mac=gen_score_dis(gf_ile1,genelist=macro,cut=ile_mac_cut)
gf_ile2_mac=gen_score_dis(gf_ile2,genelist=macro,cut=ile_mac_cut)
spf_ile2_mac=gen_score_dis(spf_ile2,genelist=macro,cut=ile_mac_cut)
spf_ile3_mac=gen_score_dis(spf_ile3,genelist=macro,cut=ile_mac_cut)
gf_col1_mac=gen_score_dis(gf_col1,genelist=macro,cut=col_mac_cut)
spf_col1_mac=gen_score_dis(spf_col1,genelist=macro,cut=col_mac_cut)
gf_cem_mac=gen_score_dis(gf_cem,genelist=macro,cut=cem_mac_cut)
spf_cem_mac=gen_score_dis(spf_cem,genelist=macro,cut=cem_mac_cut)
spf_liv_mac=gen_score_dis(spf_liv,genelist=macro,cut=liv_mac_cut)
gf_liv_mac=gen_score_dis(gf_liv,genelist=macro,cut=liv_mac_cut)
gf_thy_mac=gen_score_dis(gf_thy,genelist=macro,cut=thy_mac_cut)
spf_thy_mac=gen_score_dis(spf_thy,genelist=macro,cut=thy_mac_cut)
gf_spl_mac=gen_score_dis(gf_spl,genelist=macro,cut=spl_mac_cut)
spf_spl_mac=gen_score_dis(spf_spl,genelist=macro,cut=spl_mac_cut)

gf_ile1_mac_den=gen_score_filter(gf_ile1,genelist=macro,cut=ile_mac_cut)
gf_ile2_mac_den=gen_score_filter(gf_ile2,genelist=macro,cut=ile_mac_cut)
spf_ile2_mac_den=gen_score_filter(spf_ile2,genelist=macro,cut=ile_mac_cut)
spf_ile3_mac_den=gen_score_filter(spf_ile3,genelist=macro,cut=ile_mac_cut)
gf_col1_mac_den=gen_score_filter(gf_col1,genelist=macro,cut=col_mac_cut)
spf_col1_mac_den=gen_score_filter(spf_col1,genelist=macro,cut=col_mac_cut)
gf_cem_mac_den=gen_score_filter(gf_cem,genelist=macro,cut=cem_mac_cut)
spf_cem_mac_den=gen_score_filter(spf_cem,genelist=macro,cut=cem_mac_cut)
spf_liv_mac_den=gen_score_filter(spf_liv,genelist=macro,cut=liv_mac_cut)
gf_liv_mac_den=gen_score_filter(gf_liv,genelist=macro,cut=liv_mac_cut)
gf_thy_mac_den=gen_score_filter(gf_thy,genelist=macro,cut=thy_mac_cut)
spf_thy_mac_den=gen_score_filter(spf_thy,genelist=macro,cut=thy_mac_cut)
gf_spl_mac_den=gen_score_filter(gf_spl,genelist=macro,cut=spl_mac_cut)
spf_spl_mac_den=gen_score_filter(spf_spl,genelist=macro,cut=spl_mac_cut)



In [ ]:
gf_ile1_mac_grp=score_area_size(gf_ile1_mac,bin=20)
gf_ile2_mac_grp=score_area_size(gf_ile2_mac,bin=20)
spf_ile2_mac_grp=score_area_size(spf_ile2_mac,bin=20)
spf_ile3_mac_grp=score_area_size(spf_ile3_mac,bin=20)
gf_col1_mac_grp=score_area_size(gf_col1_mac,bin=20)
spf_col1_mac_grp=score_area_size(spf_col1_mac,bin=20)
gf_cem_mac_grp=score_area_size(gf_cem_mac,bin=20)
spf_cem_mac_grp=score_area_size(spf_cem_mac,bin=20)
gf_thy_mac_grp=score_area_size(gf_thy_mac,bin=20)
spf_thy_mac_grp=score_area_size(spf_thy_mac,bin=20)
gf_spl_mac_grp=score_area_size(gf_spl_mac,bin=20)
spf_spl_mac_grp=score_area_size(spf_spl_mac,bin=20)
spf_liv_mac_grp=score_area_size(spf_liv_mac,bin=20)
gf_liv_mac_grp=score_area_size(gf_liv_mac,bin=20)



In [ ]:
gf_ile1_mac_den_size=density_area_size(gf_ile1_mac_den,gf_ile1_mac_grp)
gf_ile2_mac_den_size=density_area_size(gf_ile2_mac_den,gf_ile2_mac_grp)
spf_ile2_mac_den_size=density_area_size(spf_ile2_mac_den,spf_ile2_mac_grp)
spf_ile3_mac_den_size=density_area_size(spf_ile3_mac_den,spf_ile3_mac_grp)
gf_col1_mac_den_size=density_area_size(gf_col1_mac_den,gf_col1_mac_grp)
spf_col1_mac_den_size=density_area_size(spf_col1_mac_den,spf_col1_mac_grp)
gf_cem_mac_den_size=density_area_size(gf_cem_mac_den,gf_cem_mac_grp)
spf_cem_mac_den_size=density_area_size(spf_cem_mac_den,spf_cem_mac_grp)
gf_thy_mac_den_size=density_area_size(gf_thy_mac_den,gf_thy_mac_grp)
spf_thy_mac_den_size=density_area_size(spf_thy_mac_den,spf_thy_mac_grp)
gf_spl_mac_den_size=density_area_size(gf_spl_mac_den,gf_spl_mac_grp)
spf_spl_mac_den_size=density_area_size(spf_spl_mac_den,spf_spl_mac_grp)
gf_liv_mac_den_size=density_area_size(gf_liv_mac_den,gf_liv_mac_grp)
spf_liv_mac_den_size=density_area_size(spf_liv_mac_den,spf_liv_mac_grp)

In [ ]:
saveRDS(gf_ile1_mac_grp,file="/data/work/7.immune_cell_aggregation_index/gf_ile1_mac_grp.txt")
saveRDS(gf_ile2_mac_grp,file="/data/work/7.immune_cell_aggregation_index/gf_ile2_mac_grp.txt")

saveRDS(spf_ile2_mac_grp,file="/data/work/7.immune_cell_aggregation_index/spf_ile2_mac_grp.txt")
saveRDS(spf_ile3_mac_grp,file="/data/work/7.immune_cell_aggregation_index/spf_ile3_mac_grp.txt")

saveRDS(gf_col1_mac_grp,file="/data/work/7.immune_cell_aggregation_index/gf_col1_mac_grp.txt")
saveRDS(spf_col1_mac_grp,file="/data/work/7.immune_cell_aggregation_index/spf_col1_mac_grp.txt")

saveRDS(gf_cem_mac_grp,file="/data/work/7.immune_cell_aggregation_index/gf_cem_mac_grp.txt")
saveRDS(spf_cem_mac_grp,file="/data/work/7.immune_cell_aggregation_index/spf_cem_mac_grp.txt")

saveRDS(gf_thy_mac_grp,file="/data/work/7.immune_cell_aggregation_index/gf_thy_mac_grp.txt")
saveRDS(spf_thy_mac_grp,file="/data/work/7.immune_cell_aggregation_index/spf_thy_mac_grp.txt")

saveRDS(gf_spl_mac_grp,file="/data/work/7.immune_cell_aggregation_index/gf_spl_mac_grp.txt")
saveRDS(spf_spl_mac_grp,file="/data/work/7.immune_cell_aggregation_index/spf_spl_mac_grp.txt")

saveRDS(spf_liv_mac_grp,file="/data/work/7.immune_cell_aggregation_index/spf_liv_mac_grp.txt")
saveRDS(gf_liv_mac_grp,file="/data/work/7.immune_cell_aggregation_index/gf_liv_mac_grp.txt")

saveRDS(gf_ile1_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_ile1_mac_den_size.txt")
saveRDS(gf_ile2_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_ile2_mac_den_size.txt")

saveRDS(spf_ile2_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_ile2_mac_den_size.txt")
saveRDS(spf_ile3_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_ile3_mac_den_size.txt")

saveRDS(gf_col1_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_col1_mac_den_size.txt")
saveRDS(spf_col1_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_col1_mac_den_size.txt")

saveRDS(gf_cem_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_cem_mac_den_size.txt")
saveRDS(spf_cem_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_cem_mac_den_size.txt")

saveRDS(gf_thy_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_thy_mac_den_size.txt")
saveRDS(spf_thy_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_thy_mac_den_size.txt")
saveRDS(gf_spl_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_spl_mac_den_size.txt")
saveRDS(spf_spl_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_spl_mac_den_size.txt")

saveRDS(gf_liv_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_liv_mac_den_size.txt")
saveRDS(spf_liv_mac_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_liv_mac_den_size.txt")

In [ ]:
nkt=c("Cd28","Cd3e","Cd3d","Trbc1","Trbc2","Trac","Foxp3","Il22","Cd4","Cd8a","Cd8b1","Ctla4","Il17a","Il13","Il4","Il9r","Il17rb","Trdc","Tcrg-C2","Tcrg-C4","Ccl4","Ccl3","Ccl5","Xcl1","Ifng","Klrd1","Klrb1","Tcf7","Ncr1","Nkg7","Gzma","Gzmb","Il2rb","Pdcd1")
gf_col1_rawnkt=gen_rawscore_dis(gf_col1,genelist=nkt)
spf_col1_rawnkt=gen_rawscore_dis(spf_col1,genelist=nkt)
gf_ile1_rawnkt=gen_rawscore_dis(gf_ile1,genelist=nkt)
gf_ile2_rawnkt=gen_rawscore_dis(gf_ile2,genelist=nkt)
spf_ile2_rawnkt=gen_rawscore_dis(spf_ile2,genelist=nkt)
spf_ile3_rawnkt=gen_rawscore_dis(spf_ile3,genelist=nkt)
gf_cem_rawnkt=gen_rawscore_dis(gf_cem,genelist=nkt)
spf_cem_rawnkt=gen_rawscore_dis(spf_cem,genelist=nkt)

spf_liv_rawnkt=gen_rawscore_dis(spf_liv,genelist=nkt)
gf_liv_rawnkt=gen_rawscore_dis(gf_liv,genelist=nkt)

gf_thy_rawnkt=gen_rawscore_dis(gf_thy,genelist=nkt)
spf_thy_rawnkt=gen_rawscore_dis(gf_thy,genelist=nkt)

gf_spl_rawnkt=gen_rawscore_dis(gf_spl,genelist=nkt)
spf_spl_rawnkt=gen_rawscore_dis(spf_spl,genelist=nkt)

cem_nkt_cut=low_percentage_identify(c(gf_cem_rawnkt[,3],spf_cem_rawnkt[,3]),low_cut=0.05)
col_nkt_cut=low_percentage_identify(c(gf_col1_rawnkt[,3],spf_col1_rawnkt[,3]),low_cut=0.05)
ile_nkt_cut=low_percentage_identify(c(gf_ile1_rawnkt[,3],gf_ile2_rawnkt[,3],spf_ile2_rawnkt[,3],spf_ile3_rawnkt[,3]),low_cut=0.05)
thy_nkt_cut=low_percentage_identify(c(gf_thy_rawnkt[,3],spf_thy_rawnkt[,3]),low_cut=0.05)
spl_nkt_cut=low_percentage_identify(c(gf_spl_rawnkt[,3],spf_spl_rawnkt[,3]),low_cut=0.05)
liv_nkt_cut=low_percentage_identify(c(gf_liv_rawnkt[,3],spf_liv_rawnkt[,3]),low_cut=0.05)



In [ ]:
cem_nkt_cut
col_nkt_cut
ile_nkt_cut
thy_nkt_cut
spl_nkt_cut
liv_nkt_cut



In [ ]:
gf_ile1_nkt=gen_score_dis(gf_ile1,genelist=nkt,cut=ile_nkt_cut)
gf_ile2_nkt=gen_score_dis(gf_ile2,genelist=nkt,cut=ile_nkt_cut)
spf_ile2_nkt=gen_score_dis(spf_ile2,genelist=nkt,cut=ile_nkt_cut)
spf_ile3_nkt=gen_score_dis(spf_ile3,genelist=nkt,cut=ile_nkt_cut)
gf_col1_nkt=gen_score_dis(gf_col1,genelist=nkt,cut=col_nkt_cut)
spf_col1_nkt=gen_score_dis(spf_col1,genelist=nkt,cut=col_nkt_cut)
gf_cem_nkt=gen_score_dis(gf_cem,genelist=nkt,cut=cem_nkt_cut)
spf_cem_nkt=gen_score_dis(spf_cem,genelist=nkt,cut=cem_nkt_cut)
spf_liv_nkt=gen_score_dis(spf_liv,genelist=nkt,cut=liv_nkt_cut)
gf_liv_nkt=gen_score_dis(gf_liv,genelist=nkt,cut=liv_nkt_cut)
gf_thy_nkt=gen_score_dis(gf_thy,genelist=nkt,cut=thy_nkt_cut)
spf_thy_nkt=gen_score_dis(spf_thy,genelist=nkt,cut=thy_nkt_cut)
gf_spl_nkt=gen_score_dis(gf_spl,genelist=nkt,cut=spl_nkt_cut)
spf_spl_nkt=gen_score_dis(spf_spl,genelist=nkt,cut=spl_nkt_cut)
gf_ile1_nkt_den=gen_score_filter(gf_ile1,genelist=nkt,cut=ile_nkt_cut)
gf_ile2_nkt_den=gen_score_filter(gf_ile2,genelist=nkt,cut=ile_nkt_cut)
spf_ile2_nkt_den=gen_score_filter(spf_ile2,genelist=nkt,cut=ile_nkt_cut)
spf_ile3_nkt_den=gen_score_filter(spf_ile3,genelist=nkt,cut=ile_nkt_cut)
gf_col1_nkt_den=gen_score_filter(gf_col1,genelist=nkt,cut=col_nkt_cut)
spf_col1_nkt_den=gen_score_filter(spf_col1,genelist=nkt,cut=col_nkt_cut)
gf_cem_nkt_den=gen_score_filter(gf_cem,genelist=nkt,cut=cem_nkt_cut)
spf_cem_nkt_den=gen_score_filter(spf_cem,genelist=nkt,cut=cem_nkt_cut)
spf_liv_nkt_den=gen_score_filter(spf_liv,genelist=nkt,cut=liv_nkt_cut)
gf_liv_nkt_den=gen_score_filter(gf_liv,genelist=nkt,cut=liv_nkt_cut)
#gf_thy_nkt_den=gen_score_filter(gf_thy,genelist=nkt,cut=thy_nkt_cut)
#spf_thy_nkt_den=gen_score_filter(spf_thy,genelist=nkt,cut=thy_nkt_cut)
gf_spl_nkt_den=gen_score_filter(gf_spl,genelist=nkt,cut=spl_nkt_cut)
spf_spl_nkt_den=gen_score_filter(spf_spl,genelist=nkt,cut=spl_nkt_cut)
gf_thy_nkt_den=gen_score_filter(gf_thy,genelist=nkt,cut=thy_nkt_cut)
spf_thy_nkt_den=gen_score_filter(spf_thy,genelist=nkt,cut=thy_nkt_cut)

In [ ]:
gf_ile1_nkt_grp=score_area_size(gf_ile1_nkt,bin=20)
gf_ile2_nkt_grp=score_area_size(gf_ile2_nkt,bin=20)
spf_ile2_nkt_grp=score_area_size(spf_ile2_nkt,bin=20)
spf_ile3_nkt_grp=score_area_size(spf_ile3_nkt,bin=20)
gf_col1_nkt_grp=score_area_size(gf_col1_nkt,bin=20)
spf_col1_nkt_grp=score_area_size(spf_col1_nkt,bin=20)
gf_cem_nkt_grp=score_area_size(gf_cem_nkt,bin=20)
spf_cem_nkt_grp=score_area_size(spf_cem_nkt,bin=20)
spf_liv_nkt_grp=score_area_size(spf_liv_nkt,bin=20)
gf_liv_nkt_grp=score_area_size(gf_liv_nkt,bin=20)
gf_spl_nkt_grp=score_area_size(gf_spl_nkt,bin=20)
spf_spl_nkt_grp=score_area_size(spf_spl_nkt,bin=20)
#gf_thy_nkt_grp=score_area_size(gf_thy_nkt,bin=20)
#spf_thy_nkt_grp=score_area_size(spf_thy_nkt,bin=20)

In [ ]:
gf_ile1_nkt_den_size=density_area_size(gf_ile1_nkt_den,gf_ile1_nkt_grp)
gf_ile2_nkt_den_size=density_area_size(gf_ile2_nkt_den,gf_ile2_nkt_grp)
spf_ile2_nkt_den_size=density_area_size(spf_ile2_nkt_den,spf_ile2_nkt_grp)
spf_ile3_nkt_den_size=density_area_size(spf_ile3_nkt_den,spf_ile3_nkt_grp)
gf_col1_nkt_den_size=density_area_size(gf_col1_nkt_den,gf_col1_nkt_grp)
spf_col1_nkt_den_size=density_area_size(spf_col1_nkt_den,spf_col1_nkt_grp)
gf_cem_nkt_den_size=density_area_size(gf_cem_nkt_den,gf_cem_nkt_grp)
spf_cem_nkt_den_size=density_area_size(spf_cem_nkt_den,spf_cem_nkt_grp)
#gf_thy_nkt_den_size=density_area_size(gf_thy_nkt_den,gf_thy_nkt_grp)
#spf_thy_nkt_den_size=density_area_size(spf_thy_nkt_den,spf_thy_nkt_grp)
gf_spl_nkt_den_size=density_area_size(gf_spl_nkt_den,gf_spl_nkt_grp)
spf_spl_nkt_den_size=density_area_size(spf_spl_nkt_den,spf_spl_nkt_grp)
gf_liv_nkt_den_size=density_area_size(gf_liv_nkt_den,gf_liv_nkt_grp)
spf_liv_nkt_den_size=density_area_size(spf_liv_nkt_den,spf_liv_nkt_grp)

In [ ]:
saveRDS(gf_ile1_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/gf_ile1_nkt_grp.txt")
saveRDS(gf_ile2_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/gf_ile2_nkt_grp.txt")

saveRDS(spf_ile2_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/spf_ile2_nkt_grp.txt")
saveRDS(spf_ile3_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/spf_ile3_nkt_grp.txt")

saveRDS(gf_col1_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/gf_col1_nkt_grp.txt")
saveRDS(spf_col1_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/spf_col1_nkt_grp.txt")

saveRDS(gf_cem_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/gf_cem_nkt_grp.txt")
saveRDS(spf_cem_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/spf_cem_nkt_grp.txt")

#saveRDS(gf_thy_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/gf_thy_nkt_grp.txt")
#saveRDS(spf_thy_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/spf_thy_nkt_grp.txt")

saveRDS(gf_spl_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/gf_spl_nkt_grp.txt")
saveRDS(spf_spl_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/spf_spl_nkt_grp.txt")

saveRDS(spf_liv_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/spf_liv_nkt_grp.txt")
saveRDS(gf_liv_nkt_grp,file="/data/work/7.immune_cell_aggregation_index/gf_liv_nkt_grp.txt")

saveRDS(gf_ile1_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_ile1_nkt_den_size.txt")
saveRDS(gf_ile2_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_ile2_nkt_den_size.txt")

saveRDS(spf_ile2_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_ile2_nkt_den_size.txt")
saveRDS(spf_ile3_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_ile3_nkt_den_size.txt")

saveRDS(gf_col1_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_col1_nkt_den_size.txt")
saveRDS(spf_col1_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_col1_nkt_den_size.txt")

saveRDS(gf_cem_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_cem_nkt_den_size.txt")
saveRDS(spf_cem_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_cem_nkt_den_size.txt")

#saveRDS(gf_thy_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_thy_nkt_den_size.txt")
#saveRDS(spf_thy_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_thy_nkt_den_size.txt")

saveRDS(gf_spl_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_spl_nkt_den_size.txt")
saveRDS(spf_spl_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_spl_nkt_den_size.txt")

saveRDS(gf_liv_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/gf_liv_nkt_den_size.txt")
saveRDS(spf_liv_nkt_den_size,file="/data/work/7.immune_cell_aggregation_index/spf_liv_nkt_den_size.txt")